In [74]:
import duckdb as ddb
import pandas as pd
import plotly.graph_objects as go

con = ddb.connect(database=":memory:")

con.sql("""
CREATE TABLE program_beslut AS
    FROM 'data/resultat-ansokningsomgang-2020-2024-beslut.csv';
CREATE TABLE program_kommun AS
    FROM 'data/resultat-ansokningsomgang-2020-2024-diarie_kommun.csv';

CREATE TABLE kurser_beslut AS
    FROM 'data/resultat-for-kurser-inom-yh-2024-beslut.csv';
CREATE TABLE kurser_kommun AS
    FROM 'data/resultat-for-kurser-inom-yh-2024-diarie_kommun.csv';

CREATE TABLE kommun_lan AS
    FROM 'data/kommunlankod-2025.csv';       
""")

In [75]:
def create_df_utbildningsomrade_beslut_single(
    con, beslut=True, utbildningsanordnare: list[str] | None = None, year=2024
):
    rel = con.sql(
        """
    select
        Utbildningsområde,
        count(*) as Beslut
    from kurser_beslut
    where
        ( $utb IS NULL OR Utbildningsanordnare = ANY($utb) )
        and Ansökningsomgång = $year
        and Beslut = $beslut
    group by Utbildningsområde
    order by Beslut desc, Utbildningsområde
    ;""",
        params={"beslut": beslut, "year": year, "utb": utbildningsanordnare},
    )

    df_rel: pd.Dataframe = rel.df().set_index("Utbildningsområde")
    return df_rel.rename(columns={"Beslut": "Beviljade" if beslut else "Avslag"})

In [76]:
def create_df_utbildningsomrade_beslut_merge(con):
    df_utbildningsomrade_beviljade = create_df_utbildningsomrade_beslut_single(con, beslut=True)
    df_utbildningsomrade_avslag = create_df_utbildningsomrade_beslut_single(con, beslut=False)

    df_utbildningsomrade_beslut = pd.merge(
        df_utbildningsomrade_beviljade,
        df_utbildningsomrade_avslag,
        left_index=True,
        right_index=True,
    )

    df_utbildningsomrade_beslut["diff_beslut"] = abs(
        df_utbildningsomrade_beslut["Beviljade"] - df_utbildningsomrade_beslut["Avslag"]
    )

    return df_utbildningsomrade_beslut

In [77]:
def create_df_utbildningsomrade_beslut(
    con,
    utbildningsanordnare: list[str] | None = None,
    year=2024,
):
    """
    Filter by one or many Utbildningsanordnare.
    """

    rel = con.sql(
        """
    select
        Utbildningsområde,
        sum(case when Beslut = TRUE then 1 else 0 end)::integer as Beviljade,
        sum(case when Beslut = FALSE then 1 else 0 end)::integer as Avslag,
        abs(Beviljade - Avslag) as diff_beslut
    from kurser_beslut
    where
        ( $utb IS NULL OR Utbildningsanordnare = ANY($utb) )
        and Ansökningsomgång = $year
    group by Utbildningsområde
    order by (Beviljade + Avslag) desc, Utbildningsområde
    ;""",
        params={"year": year, "utb": utbildningsanordnare},
    )

    return rel.df().set_index("Utbildningsområde")


create_df_utbildningsomrade_beslut(con)

,Beviljade,Avslag,diff_beslut
Utbildningsområde,,,
"Ekonomi, administration och försäljning",171,112,59
Data/IT,111,84,27
Teknik och tillverkning,106,51,55
Hälso- och sjukvård samt socialt arbete,81,53,28
Samhällsbyggnad och byggteknik,93,41,52
"Lantbruk, djurvård, trädgård, skog och fiske",18,24,6
Pedagogik och undervisning,15,18,3
"Hotell, restaurang och turism",19,9,10
Säkerhetstjänster,11,11,0


In [78]:
def make_fig_utbildningsomrade_beslut(df_utbildningsomrade_beslut):
    fig = go.Figure()

    fig.add_trace(
        go.Bar(
            y=df_utbildningsomrade_beslut.index,
            x=df_utbildningsomrade_beslut["Beviljade"],
            name="Beviljade",
            hovertemplate="%{x}<extra>Beviljade</extra>",
            orientation="h",
            text=df_utbildningsomrade_beslut["Beviljade"],
            textposition="inside",
            insidetextanchor="end",
            textangle=0,
            marker=dict(color="rgba(246, 78, 139, 0.6)"),
        )
    )

    fig.add_trace(
        go.Bar(
            y=df_utbildningsomrade_beslut.index,
            x=df_utbildningsomrade_beslut["diff_beslut"],
            name="Avslag",
            customdata=df_utbildningsomrade_beslut["Avslag"],
            hovertemplate="%{customdata}<extra>Avslag</extra>",
            orientation="h",
            text=df_utbildningsomrade_beslut["Avslag"],
            textposition="inside",
            insidetextanchor="end",
            textangle=0,
            marker=dict(color="rgba(58, 71, 80, 0.6)"),
        )
    )

    sort_order = df_utbildningsomrade_beslut.sort_values("Beviljade").index

    fig.update_layout(
        barmode="stack",
        # yaxis={"categoryorder": "total ascending"},
        yaxis={"categoryorder": "array", "categoryarray": sort_order},
        legend=dict(traceorder="normal"),
    )

    return fig


df_utbildningsomrade_beslut = create_df_utbildningsomrade_beslut(con)
fig_utbildningsomrade_beslut = make_fig_utbildningsomrade_beslut(df_utbildningsomrade_beslut)

fig_utbildningsomrade_beslut

In [79]:
def make_fig_utbildningsomrade_beslut(df_utbildningsomrade_beslut):
    fig = go.Figure()

    fig.add_trace(
        go.Bar(
            y=df_utbildningsomrade_beslut.index,
            x=df_utbildningsomrade_beslut["Avslag"],
            name="Avslag",
            customdata=df_utbildningsomrade_beslut["Avslag"],
            hovertemplate="%{customdata}<extra>Avslag</extra>",
            orientation="h",
            text=df_utbildningsomrade_beslut["Avslag"],
            textposition="inside",
            insidetextanchor="end",
            textangle=0,
            marker=dict(color="rgba(58, 71, 80, 0.6)"),
        )
    )

    fig.add_trace(
        go.Bar(
            y=df_utbildningsomrade_beslut.index,
            x=df_utbildningsomrade_beslut["Beviljade"],
            name="Beviljade",
            hovertemplate="%{x}<extra>Beviljade</extra>",
            orientation="h",
            text=df_utbildningsomrade_beslut["Beviljade"],
            textposition="inside",
            insidetextanchor="end",
            textangle=0,
            marker=dict(color="rgba(246, 78, 139, 0.6)"),
        )
    )

    sort_order = df_utbildningsomrade_beslut.sort_values("Beviljade").index

    fig.update_layout(
        # barmode="stack",
        yaxis={"categoryorder": "array", "categoryarray": sort_order},
        legend=dict(traceorder="normal"),
    )

    return fig


df_utbildningsomrade_beslut = create_df_utbildningsomrade_beslut(con)
fig_utbildningsomrade_beslut = make_fig_utbildningsomrade_beslut(df_utbildningsomrade_beslut)

fig_utbildningsomrade_beslut

In [80]:
def make_fig_utbildningsomrade_beslut(df_utbildningsomrade_beslut):
    fig = go.Figure()

    fig.add_trace(
        go.Bar(
            y=df_utbildningsomrade_beslut.index,
            x=df_utbildningsomrade_beslut["Avslag"],
            name="Avslag",
            # customdata=df_utbildningsomrade_beslut["Avslag"],
            # hovertemplate="%{customdata}<extra>Avslag</extra>",
            hovertemplate="%{x}<extra>Avslag</extra>",
            orientation="h",
            text=df_utbildningsomrade_beslut["Avslag"],
            textposition="inside",
            insidetextanchor="end",
            textangle=0,
            marker=dict(color="rgba(58, 71, 80, 0.6)"),
        )
    )

    fig.add_trace(
        go.Bar(
            y=df_utbildningsomrade_beslut.index,
            x=df_utbildningsomrade_beslut["Beviljade"] - df_utbildningsomrade_beslut["Avslag"],
            name="Beviljade",
            customdata=df_utbildningsomrade_beslut["Beviljade"],
            hovertemplate="%{customdata}<extra>Beviljade</extra>",
            # hovertemplate="%{x}<extra>Beviljade</extra>",
            orientation="h",
            text=df_utbildningsomrade_beslut["Beviljade"],
            textposition="outside",
            insidetextanchor="end",
            textangle=0,
            marker=dict(color="rgba(246, 78, 139, 0.6)"),
        )
    )

    sort_order = df_utbildningsomrade_beslut.sort_values("Beviljade").index

    fig.update_layout(
        barmode="stack",
        # yaxis={"categoryorder": "total ascending"},
        yaxis={"categoryorder": "array", "categoryarray": sort_order},
        legend=dict(traceorder="normal"),
    )

    return fig


df_utbildningsomrade_beslut = create_df_utbildningsomrade_beslut(con)
fig_utbildningsomrade_beslut = make_fig_utbildningsomrade_beslut(df_utbildningsomrade_beslut)

fig_utbildningsomrade_beslut